In [20]:
import pandas as pd
import re 
import string
from tqdm.auto import tqdm 
tqdm.pandas()

import logging
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis.gensim_models
import pyLDAvis

from nltk.corpus import stopwords

import plotly.express as px


In [17]:
df_org = pd.read_parquet("../data/processed/pyarrow/UfR_text.parquet")
df = df_org.copy()

In [3]:
stop = stopwords.words('danish')

def preprocessing(s):
  if pd.isna(s): 
    return []
  else:
    #remove \n newline character
    s = re.sub("\\n", ' ', s)
    #remove digits
    s = re.sub('[0-9]', ' ', s)
    # lowercase
    s = s.lower() 
    #remove punctuation
    s = s.translate(str.maketrans('', '', string.punctuation))
    #tokenise
    tokens = s.split()
    #remove empty
    tokens = list(filter(None, tokens))
    # #remove oneletter words
    tokens = [x for x in tokens if len(x)>1]
    # #remove stopwords
    tokens = [item for item in tokens if item not in stop]
    return tokens

df["tokens"] = df["verdict_text"].progress_apply(preprocessing)

100%|██████████| 63915/63915 [04:18<00:00, 247.36it/s]


In [4]:
all_tokens = [x for x in df["tokens"].to_list() if x is not None]

In [5]:
id2word = Dictionary(all_tokens) 
id2word.filter_extremes(no_below=7, no_above=.2, keep_n = None)
corpus = [id2word.doc2bow(doc) for doc in tqdm(all_tokens)]


100%|██████████| 63915/63915 [00:38<00:00, 1658.77it/s]


In [46]:
logging.basicConfig(filename='gensim.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.INFO)

In [47]:
num_topics = 20
lda_model = LdaMulticore(corpus=corpus, num_topics=20, id2word=id2word, passes = 6, iterations = 400, eval_every=1)
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(LDAvis_prepared, f"../data/plots/ldavis_num_topics_{num_topics}.html")
LDAvis_prepared


C:\Users\espen\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15    -0.139507 -0.114030       1        1  18.015533
18    -0.120967 -0.061409       2        1  12.139092
8     -0.144756  0.108240       3        1   8.616524
7      0.008040 -0.127876       4        1   7.693411
11    -0.152156 -0.018267       5        1   7.035252
6     -0.053297  0.168721       6        1   5.436102
13    -0.004847 -0.066415       7        1   5.131441
12     0.037121 -0.039991       8        1   3.676211
3      0.091498 -0.017714       9        1   3.447646
16     0.183394  0.007249      10        1   3.408166
0     -0.080902  0.053828      11        1   3.400614
4      0.109406  0.134647      12        1   3.214264
19     0.115974 -0.059007      13        1   3.210622
10    -0.002746  0.012890      14        1   2.841878
2      0.041907 -0.087688      15        1   2.642322
9      0.001772 -0.008389      16        1   2.599342
17     0.027694  0.069190      17        1   2.538581
5     -0.085871  0.039282      18        1   2.184954
14     0.067159 -0.058805      19        1   1.783090
1      0.101081  0.065544      20        1   0.984957, topic_info=                Term          Freq         Total Category  logprob  loglift
9342             the  55752.000000  55752.000000  Default  30.0000  30.0000
12662     ejendommen  95608.000000  95608.000000  Default  29.0000  29.0000
9898              of  31340.000000  31340.000000  Default  28.0000  28.0000
787    straffelovens  66084.000000  66084.000000  Default  27.0000  27.0000
1415      sagsøgerne  75203.000000  75203.000000  Default  26.0000  26.0000
...              ...           ...           ...      ...      ...      ...
627      christensen    727.797751  27964.114672  Topic20  -6.3433   0.9717
253           stande    629.699464  13700.600160  Topic20  -6.4881   1.5404
180       offentlige    659.192297  21949.867419  Topic20  -6.4424   1.1148
1900             pct    683.702691  39379.720870  Topic20  -6.4058   0.5668
536           saadan    640.595494  19348.504414  Topic20  -6.4710   1.2124

[1925 rows x 6 columns], token_table=        Topic      Freq         Term
term                                
43474       3  0.062194    aaleruser
43474      10  0.093292    aaleruser
43474      12  0.777430    aaleruser
43474      13  0.031097    aaleruser
95141       4  0.978613       aamann
...       ...       ...          ...
39646      18  0.985091     østerild
103559      3  0.993817  østjyllands
51774       7  0.218921           …«
51774      11  0.660180           …«
51774      16  0.120703           …«

[18578 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 19, 9, 8, 12, 7, 14, 13, 4, 17, 1, 5, 20, 11, 3, 10, 18, 6, 15, 2])

In [48]:
get_document_topics = [lda_model.get_document_topics(item) for item in tqdm(corpus)]

100%|██████████| 63915/63915 [02:00<00:00, 531.60it/s] 


In [54]:
df["topics"] = get_document_topics
df_with_topics = pd.merge(df,pd.DataFrame(df["topics"].explode().tolist(), index=df["topics"].explode().index, columns=["topic_id","distribution"]),left_index=True,right_index=True)
df_with_topics["verdict_index"]=df_with_topics.index
df_with_topics = df_with_topics.sort_values(["verdict_index","distribution"],ascending=False).drop_duplicates("verdict_index")
df_with_topics = df_with_topics.rename(columns={"topic_id":"most_prevalent_topic","distribution":"prevalence_of_topic"})

In [9]:
ct_topic_year = df_with_topics.groupby(["year","most_prevalent_topic"]).count()["id_verdict"].reset_index()
verdicts_pr_year = ct_topic_year.groupby("year").sum().rename(columns={"id_verdict":"total_verdicts_in_year"}).reset_index()

In [27]:
ct_topic_year_share = pd.merge(ct_topic_year, verdicts_pr_year.loc[:,["year","total_verdicts_in_year"]], left_on="year", right_on="year")
ct_topic_year_share["topic_proportion"] = ct_topic_year_share["id_verdict"]/ct_topic_year_share["total_verdicts_in_year"]

ct_topic_year_share["year"] = ct_topic_year_share["year"].astype(int)
ct_topic_year_share["most_prevalent_topic"] = (ct_topic_year_share["most_prevalent_topic"].astype(int)+1).astype(str)
# Create bar chart displaying the evolution of headers
fig = px.bar(ct_topic_year_share, x="year",y="topic_proportion",color="most_prevalent_topic")
fig.update_yaxes(tickformat=".0%", range=[0,1])
fig

In [69]:

df_with_topics.to_parquet(f"../data/processed/pyarrow/UfR_with_LDA_topics_num_topics_{num_topics}.parquet")